In [7]:
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import scipy.stats as stats
import scipy.optimize as opt

In [8]:
data = np.genfromtxt("metals.dat", names=True, dtype=None)

# print out all columns we just got for free
data.dtype

dtype([('R_GC', '<f8'), ('FE_H', '<f8'), ('FE_H_ERR', '<f8'), ('O_FE', '<f8'), ('O_FE_ERR', '<f8')])

In [9]:
# let's make a function the normal way
def linear(x,m,b):
    return m*x+b

In [33]:
# note, result is dependent on choice of loss function
# need to see method to trf in order to use least_squares instead of leastsq
popt, pcov = opt.curve_fit(linear, data['R_GC'],data['O_FE'],method='trf',loss='arctan',f_scale=0.20)

In [34]:
print (*popt)

0.006766707931389952 -0.06938331415905287


In [32]:
popt_abs,pcov = opt.curve_fit(linear, data['R_GC'],data['O_FE'],sigma=data['O_FE_ERR'],absolute_sigma=True)
print (*popt_abs)

popt_rel,pcov = opt.curve_fit(linear, data['R_GC'],data['O_FE'],sigma=data['O_FE_ERR'],absolute_sigma=False)
print (*popt_rel)

0.005358711605572734 -0.058178020691403
0.005358711605572734 -0.058178020691403


In [ ]:
# add x error and monty carlo (and it would be nice to make their errorbars prettier)